In [51]:
import platform #os 정보를 가져 올 수 있는 모듈

from matplotlib import font_manager, rc # font_manager : 폰트 관리 모듈, rc : 폰트 변경 모듈
from matplotlib import pyplot as plt # 시각화 도구

import seaborn as sns 

%matplotlib inline
plt.rcParams['axes.unicode_minus'] = False # 유니코드 표현 설정하기


if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') # OS가 APPLE(Darwin)이면 AppleGothic으로 폰트 설정
elif platform.system() == 'Windows':
    path="c:/Windows/Fonts/malgun.ttf" #OS가 윈도우즈면 맑은 고딕으로 설정
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown System... sorry~~~~')

In [52]:
from bs4 import BeautifulSoup # HTML 문자.te열을 DOM 구조화
from urllib.request import urlopen # url을 이용한 http 요청을 하기위한 객체
from urllib.parse import urljoin # url을 굉장히 다루기 쉽게 해준다. 

import pandas as pd

## urljoin을 이용해서 크롤링 대상 url 만들기

In [53]:
url_base = 'https://movie.naver.com/'
url_sub = 'https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=pnt&date=20200715'

url = urljoin(url_base, url_sub)
url

'https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=pnt&date=20200715'

## 영화 평점 랭킹 페이지 가져오기

In [54]:
page = urlopen(url)
soup = BeautifulSoup(page, 'html.parser')

## 영화 평점 페이지 확인 결과 
- 테이블 중간 중간 비어있는 항목이 있었다. 
 - 따라서 tbody 안이 tr만 사용해서 스크래이핑 할 때 에외 처리가 반드시 필요할 것 같다. 
 
- 직접 랭킹, 제목, 평점을 선택해서 가지고 올 것임

In [55]:
soup.find('td', class_='title').text.strip()

'그린 북'

In [56]:
td_ac = soup.find('td', class_='ac')
img_ac = td_ac.find('img')
img_ac.get('alt')

'01'

In [57]:
soup.find('td', class_='point').text.strip()

'9.59'

In [58]:
acList = []
titleList = []
pointList = []

for movie_td in soup.find_all('td', class_='title'):
    title = movie_td.text.strip()
    titleList.append(title)
    
for movie_td in soup.find_all('td', class_='ac'):
    if(movie_td.find('img') is not None and movie_td.find('img') is not 'na'):
        ac = movie_td.find('img').get('alt').strip()
    acList.append(ac)
    
for movie_td in soup.find_all('td', class_='point'):
    point = movie_td.text.strip()
    pointList.append(point)

In [59]:
acList

['01',
 '01',
 'na',
 'na',
 '02',
 '02',
 'na',
 'na',
 '03',
 '03',
 'na',
 'na',
 '04',
 '04',
 'na',
 'na',
 '05',
 '05',
 'na',
 'na',
 '06',
 '06',
 'na',
 'na',
 '07',
 '07',
 'na',
 'na',
 '08',
 '08',
 'na',
 'na',
 '09',
 '09',
 'na',
 'na',
 '010',
 '010',
 'na',
 'na',
 '11',
 '11',
 'na',
 'na',
 '12',
 '12',
 'na',
 'na',
 '13',
 '13',
 'na',
 'na',
 '14',
 '14',
 'na',
 'na',
 '15',
 '15',
 'na',
 'na',
 '16',
 '16',
 'na',
 'na',
 '17',
 '17',
 'na',
 'na',
 '18',
 '18',
 'na',
 'na',
 '19',
 '19',
 'na',
 'na',
 '20',
 '20',
 'na',
 'na',
 '21',
 '21',
 'na',
 'na',
 '22',
 '22',
 'up',
 'up',
 '23',
 '23',
 'down',
 'down',
 '24',
 '24',
 'na',
 'na',
 '25',
 '25',
 'na',
 'na',
 '26',
 '26',
 'na',
 'na',
 '27',
 '27',
 'na',
 'na',
 '28',
 '28',
 'na',
 'na',
 '29',
 '29',
 'na',
 'na',
 '30',
 '30',
 'na',
 'na',
 '31',
 '31',
 'na',
 'na',
 '32',
 '32',
 'na',
 'na',
 '33',
 '33',
 'na',
 'na',
 '34',
 '34',
 'na',
 'na',
 '35',
 '35',
 'na',
 'na',
 '36',
 '36',


---
여기까지 내 코드 다음부터 강사님 코드

In [60]:
print(soup.find('td', class_='ac').find('img')['alt'])
print(soup.find('div', class_='tit5').text)
print(soup.find('td', class_='point').text.strip())

01

그린 북

9.59


크롤링 할 때 공백제거 이외의 특별한 로직에 의해서 수집이 되어야 하면 for 및 zip 문법을 상ㅇ하고, 그 외의 경우는 컴프리핸션을 사용

In [61]:
# 컴프리핸션을 사용하는 경우 

ranks = [ td.find('img') for td in soup.find_all('td', class_='ac') if td.find('img') is not None ]
ranks = [rank.get('alt').strip() for rank in ranks if (rank.get('alt') != 'na' and rank.get('alt') != 'up' and rank.get('alt') != 'down')]
movies = [movie.text.strip() for movie in soup.find_all('div', class_='tit5')]
points = [point.text.strip() for point in soup.find_all('td', class_='point')]

## 컴프리핸션을 이용하지 않는 경우 
rank_list = []
movie_list = []
point_list = []

for rank, movie, point in zip(ranks, movies, points): pass

In [62]:
df_movie_rank = pd.DataFrame({
    '랭킹':ranks,
    '영화':movies,
    '평점':points
    
})
df_movie_rank.set_index('랭킹', inplace=True)

In [63]:
df_movie_rank.head()

,영화,평점
랭킹,,
01,그린 북,9.59
02,가버나움,9.59
03,베일리 어게인,9.53
04,원더,9.49
05,포드 V 페라리,9.49


# 100일간 네이버 영화의 평점의 변화를 확인하자 
- 100일치의 날짜 데이터를 생성 
- pandas의 date_range

In [64]:
date = pd.date_range('2020-04-07', periods=100, freq='D')
date

DatetimeIndex(['2020-04-07', '2020-04-08', '2020-04-09', '2020-04-10',
               '2020-04-11', '2020-04-12', '2020-04-13', '2020-04-14',
               '2020-04-15', '2020-04-16', '2020-04-17', '2020-04-18',
               '2020-04-19', '2020-04-20', '2020-04-21', '2020-04-22',
               '2020-04-23', '2020-04-24', '2020-04-25', '2020-04-26',
               '2020-04-27', '2020-04-28', '2020-04-29', '2020-04-30',
               '2020-05-01', '2020-05-02', '2020-05-03', '2020-05-04',
               '2020-05-05', '2020-05-06', '2020-05-07', '2020-05-08',
               '2020-05-09', '2020-05-10', '2020-05-11', '2020-05-12',
               '2020-05-13', '2020-05-14', '2020-05-15', '2020-05-16',
               '2020-05-17', '2020-05-18', '2020-05-19', '2020-05-20',
               '2020-05-21', '2020-05-22', '2020-05-23', '2020-05-24',
               '2020-05-25', '2020-05-26', '2020-05-27', '2020-05-28',
               '2020-05-29', '2020-05-30', '2020-05-31', '2020-06-01',
      

tqdm_notebook을 활용해서 진행 상황 시각화
- pip(pip3) install tqdm

In [65]:
import tqdm
import urllib

In [66]:
movie_date = [] # 평점 검색 날짜
movie_name = [] # 영화 제목
movie_point = [] # 해당 일자의 영화 평점 점수

In [67]:
url_base = 'https://movie.naver.com/'
url_sub = 'movie/sdb/rank/rmovie.nhn?sel=pnt&date={}'

In [78]:
for today in tqdm.notebook.tqdm(date):
    # yyyy-mm-dd 형식의 날짜 형식을 yyyymmdd 형식으로 바꿔주기
    today = urllib.parse.quote(today.strftime('%Y%m%d'))
    url_formated = url_sub.format(today)
    url = urljoin(url_base, url_formated)
    
    page = urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    
    # 조회한 날짜마다의 영화의 제목과 평점을 한꺼번에 구하기 
    movies = [movie.text.strip() for movie in soup.find_all('div', class_='tit5')]
    points = [float(point.text.strip()) for point in soup.find_all('td', class_='point')]
    
    movie_date.extend([ today for n in range(0, len(movies))]) # 날짜 데이터 리스트 추가 
    movie_name.extend(movies)
    movie_point.extend(points)

In [76]:
movie_df = pd.DataFrame({
    'date':movie_date,
    'name':movie_name,
    'point':movie_point
})

movie_df.head()

,date,name,point
0,20200407,그린 북,9.60
1,20200407,가버나움,9.58
2,20200407,베일리 어게인,9.52
3,20200407,포드 V 페라리,9.50
4,20200407,주전장,9.50


## 100일간의 평균 평점이 높은 순으로 정렬하기 

In [15]:
import numpy as np

In [77]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    10000 non-null  object
 1   name    10000 non-null  object
 2   point   10000 non-null  object
dtypes: object(3)
memory usage: 234.5+ KB


In [72]:
movie_agg = pd.pivot_table[movie_df, index=['name'], values='point', aggfunc=np.mean]
movie_best = movie_agg.sort_values(by='point', ascending=False)



SyntaxError: invalid syntax (<ipython-input-72-73864c89a9da>, line 1)

## 영화별 평점 확인하기 

In [75]:
movie_pivot = pd.pivot_table(movie_df, index=['date'], columns=['name'], values=['point'])
movie_pivot.head()

DataError: No numeric types to aggregate

5rows x 51 columns

pivot_table을 활용하면 컬럼의 변화가 일어났을 때 원래 있던 컬럼 때문에 컬럼의 레벨이 올라가면 droplevel()을 활용하여 상위 컬럼 레벨을 하나씩 줄여갈 수 있다. 

In [ ]:
movie_pivot.columns = movie_pivot.columns.droplevel()
movie_pivot.head()